## Loading Dataset

In [1]:
from IndexDataset import IndexDataset
import pandas as pd
from torch.utils.data import DataLoader

dataset_name = "data/imdb/imdb_top_1000.csv"
column_name = "Overview"
batch_size = 8

df = pd.read_csv(dataset_name)
dataset = IndexDataset(df, column_name)
dataloader = DataLoader(dataset, batch_size=batch_size)

## Training Loop

In [15]:
import lightning as L
import torch
from torch import nn, optim
import torch.nn.functional as F

class LitIndexer(L.LightningModule):
    def __init__(self, mapper, indexer):
        super().__init__()
        self.mapper = mapper
        self.indexer = indexer
        
    def loss(self, pred_idxs, real_idxs):
        return F.mse_loss(pred_idxs, real_idxs)

    def training_step(self, batch, batch_idx):
        strs, idxs = batch
        mapped_strs = self.mapper.forward(strs)
        pred_idxs = self.indexer(mapped_strs)
        loss = self.loss(pred_idxs, idxs.to(torch.float32))
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        strs, idxs = batch
        mapped_strs = self.mappers(strs)
        pred_idxs = self.indexer(mapped_strs)
        loss = self.loss(pred_idxs, idxs)
        self.log("test_loss", loss, prog_bar=True)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [16]:
from models.BaselineHash import BaselineHash
from models.LinearModel import LinearRegressionModel
import mmh3

hash = mmh3.hash    # 32-bits
mapper = BaselineHash(dataset, hash)
indexer = LinearRegressionModel(1)


model = LitIndexer(mapper, indexer)

trainer = L.Trainer(accelerator="cpu")
trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.

  | Name    | Type                  | Params
--------------------------------------------------
0 | indexer | LinearRegressionModel | 2     
--------------------------------------------------
2         Trainable params
0         Non-trainable params
2         Total params
0.000     Total estimated model params size (MB)
/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors

Training: |          | 0/? [00:00<?, ?it/s]

/var/folders/9c/5hbdnj_57xv861zqcz43bs6w0000gn/T/ipykernel_12211/3067490398.py:13: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred_idxs, real_idxs)
/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
trainer.test(model, dataloaders=dataloader)